In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import (
    Dense,
    Input,
    LSTM,
    Embedding,
    Dropout,
    Activation,
    SpatialDropout1D
)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from ast import literal_eval
from sklearn.model_selection import train_test_split
from tensorflow.python.keras import backend as K


sess = tf.Session()
K.set_session(sess)

In [2]:
df = pd.read_csv('../data/pandas_data_frame.csv', index_col=0)
all_data = df.where((pd.notnull(df)), '')
all_data['hashtag'] = all_data['hashtag'].apply(literal_eval)

full_text = all_data['tidy_tweet'][(all_data['label']=='1.0') | (all_data['label']=='0.0')]
y = all_data['label'][(all_data['label']=='1.0') | (all_data['label']=='0.0')]

In [3]:
max_len = 120

texts = full_text.tolist()
texts = [' '.join(t.split()[:max_len]) for t in texts]
texts = np.array(texts, dtype=object)[:, np.newaxis]

In [4]:
x_train, x_val, y_train, y_val = train_test_split(texts, y, random_state=1992, test_size=0.2)

print(x_train.shape,y_train.shape)
print(x_val.shape,y_val.shape)

(25569, 1) (25569,)
(6393, 1) (6393,)


In [5]:
import sys
sys.path.append("../") 
from personal_library.sce_keras.loss_functions import f1_loss
from personal_library.sce_keras.metrics_functions import f1
from personal_library.sce_keras.layers.elmo import ElmoEmbeddingLayer
from personal_library.sce_keras.callbacks import (
    LearningRateDecay,
    WarmUpCosineDecayScheduler
)


num_classes = 1
batch_size = 32
epochs = 2
learnRate = 0.001
lstm_out = 200
warmup_epoch = 1

warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base=learnRate,
                                        warmup_learning_rate=0,
                                        warmup_epoch=warmup_epoch,
                                        hold_base_rate_steps=5,
                                        verbose=0)

checkpoint_path = "../model_wehigts/8_w.hdf5"
checkpoint_path1 = "../model_wehigts/8_ch.hdf5"
checkpointer = ModelCheckpoint(filepath=checkpoint_path,
                               monitor='val_loss', verbose=2,
                               save_best_only=True, mode='min')
checkpointer1 = ModelCheckpoint(filepath=checkpoint_path1,
                               monitor='val_loss', verbose=2,
                               save_best_only=False, mode='min')

input_text = Input(shape=(1,), dtype="string", name='input_0')
x = ElmoEmbeddingLayer(trainable=False)(input_text)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dense(num_classes, activation="sigmoid")(x)

model = Model(inputs=[input_text], outputs=x)
model.summary()

# 'binary_crossentropy'
model.compile(loss='binary_crossentropy',#f1_loss, 
              optimizer='adam', 
              metrics=['accuracy', f1])

sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

history = model.fit(x_train, y_train, 
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[checkpointer, checkpointer1, warm_up_lr]
                   )

W0708 20:01:55.990183 140736058794880 deprecation.py:506] From /Users/sebastiancorrea/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_0 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
elmo_embedding_layer (ElmoEm (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               262400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 262,657
Trainable params: 262,657
Non-trainable params: 0
_________________________________________________________________


W0708 20:01:56.315850 140736058794880 deprecation.py:323] From /Users/sebastiancorrea/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25569 samples, validate on 6393 samples
Epoch 1/2
 1888/25569 [=>............................] - ETA: 22:21 - loss: 0.7662 - acc: 0.4571 - f1: 0.1184

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

#Load best model
model.load_weights(checkpoint_path)
y_pred = model.predict(x_val, batch_size=1)
y_pred = np.where(y_pred > 0.5, 1, 0)

print("Own emmbeding f1_sklearn: {}".format(f1_score(y_val.astype(float), y_pred)))
print("Own emmbeding accuracy: {}".format(accuracy_score(y_val.astype(float), y_pred)))